In [13]:
import sys
import os
import codecs
from math import pow, sqrt

In [14]:
users = {}
movies = {}
base_file = 'u1.base'
file = open(base_file,'r')
for line in file.readlines():
    line = line.strip().split()
    if not line[0] in users.keys():
        users[line[0]] = {line[1]:line[2]}
    else:
        users[line[0]][line[1]] = line[2]
        
    if not line[1] in movies.keys():
        movies[line[1]] = {line[0]:line[2]}
    else:
        movies[line[1]][line[0]] = line[2]

In [15]:
test = {}
ratesums = 0
test_file = 'u1.test'
file = open(test_file,'r')
for line in file.readlines():
    line = line.strip().split()
    if not line[0] in test.keys():
        test[line[0]] = {line[1]:line[2]}
    else:
        test[line[0]][line[1]] = line[2]

In [16]:
item = {}
mv_type = []

type_file = 'u.genre'
t_file = open(type_file,"r")
for t in t_file.readlines():
    t = t.strip().split('|')
    mv_type.append(t[0])
mv_type.pop()

item_file = 'u.item'
files = open(item_file, "r",  encoding="iso-8859-15")
for line in files.readlines():
    line = line.strip().split('|')
    item[line[0]] = [line[1], line[2],line[4]]
    type_list = line[5:]
    for i in range(len(mv_type)):
        if type_list[i] == '1':
            item[line[0]].append(mv_type[i])
        

In [17]:
def sim_cosine(prefs,item1, item2):
    si = {}
    for i in prefs[item1]:
        if i in prefs[item2]:
            si[i] = 1
    
    n = len(si)
    if n == 0: 
        return 0

    x = sqrt(sum([float(prefs[item1][it]) ** 2 for it in si]))
    y = sqrt(sum([float(prefs[item2][it]) ** 2 for it in si]))
    xy = sum([float(prefs[item1][it]) * float(prefs[item2][it]) for it in si])
    cos = xy / (x * y)

    return cos

In [18]:
def sim_cosine_imp(prefs, item1, item2):
    si = {}
    sum_item1 = 0.0
    sum_item2 = 0.0
    for i in prefs[item1]:
        if i in prefs[item2]:
            sum_item1 += float(prefs[item1][i])
            sum_item2 += float(prefs[item2][i])
            si[i] = 1

    n = len(si)
    if n == 0: 
        return 0
    else:
        ava1 = sum_item1/n
        ava2 = sum_item2/n
    
    x = sqrt(sum([(float(prefs[item1][it]) - ava1) ** 2 for it in si]))
    y = sqrt(sum([(float(prefs[item2][it]) - ava2) ** 2 for it in si]))
    if x + y == 0.0:
        return 1.0
    elif x * y == 0.0:
        return 0.0
    else:
        xy = sum([(float(prefs[item1][it]) - ava1) * (float(prefs[item2][it]) - ava2) for it in si])
        cos = xy / (x * y)
        return cos


In [19]:
def simularity(data,user1,user2):
    
    user1_data = data[user1]
    user2_data = data[user2]
    distance = 0.0
    for key in user1_data.keys():
        if key in user2_data.keys():
            distance += pow(float(user1_data[key])-float(user2_data[key]),2)
 
    return 1/(1+sqrt(distance))

In [22]:
def top_simliar(userId):
    
    res = {}
    movie = users[userId]
    if len(movie) == 0:
        return []
    nbhd = []
    for mv in movie:
        for pp in movies[mv]:
            nbhd.append(pp)
    nbhd = list(set(nbhd))
    for user in nbhd:
        res[user] = simularity(users,userId,user)
    
    res = sorted(res, key=lambda key: res[key],reverse=True)
    
    return res[:4]

In [24]:
def predict_rating(userId,movieId):
    
    sim_list = top_simliar(userId)
    if movieId not in movies:
        return 3.0
    if sim_list == []:
        return 3.0
    simsums = 0.0
    predict = 0.0
    
    for u in sim_list:
        if movieId in users[u]:
            simsums += simularity(users, u, userId)
        
    if simsums == 0.0:
        n = 0
        r_sums = 0.0
        for r in movies[movieId]:
            r_sums += float(movies[movieId][r])
            n += 1
        predict = float(round(r_sums/n))
    else:
        for u in sim_list:
            if movieId in users[u]:
                sim = simularity(users, u, userId)
                predict += sim / simsums * float(users[u][movieId])
    return predict

In [25]:
predict_rating('1','9')

5.0

In [26]:
def pre_rate_evaluation(userId):
    
    ratesums = 0  
    for i in test[userId]:
        rate = abs(float(predict_rating(userId,i)) - float(test[userId][i])) / float(test[userId][i])
        rate = 1 - rate
        ratesums += rate
        
    return ratesums / len(test[userId])

In [30]:
print('{:.2%}'.format(pre_rate_evaluation('1')))

63.99%


In [29]:
sums = 0.0
for i in test:
    sums += pre_rate_evaluation(i)
print('{:.2%}'.format(sums/len(test)))

67.08%
